In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import scipy.signal as signal
import arviz as az
from scipy.sparse import lil_matrix

In [3]:
# 固定图像尺寸
IMAGE_SIZE = 32
PSF_SIZE = 9

# 生成清晰图像
np.random.seed(42)
true_image = np.zeros((IMAGE_SIZE, IMAGE_SIZE))
true_image[10:20, 10:20] = 1.0

# 生成高斯 PSF
def gaussian_kernel(size, sigma):
    """生成二维高斯核"""
    kernel = np.fromfunction(
        lambda x, y: (1/(2*np.pi*sigma**2)) * np.exp(-((x-(size-1)/2)**2 + (y-(size-1)/2)**2)/(2*sigma**2)),
        (size, size)
    )
    return kernel / np.sum(kernel)  # 归一化

psf = gaussian_kernel(PSF_SIZE, 2.0)

# 生成模糊图像
blurred = signal.convolve2d(true_image, psf, mode='same')
noisy_image = np.random.poisson(blurred * 10)

In [4]:
def build_conv_matrix(psf, image_size):
    """将卷积操作转换为稀疏矩阵乘法"""
    pad = PSF_SIZE // 2
    padded_size = image_size + 2 * pad
    conv_matrix = lil_matrix((image_size**2, image_size**2))
    
    for i in range(image_size):
        for j in range(image_size):
            # 每个输出像素对应一个卷积窗口
            row_idx = i * image_size + j
            for ki in range(PSF_SIZE):
                for kj in range(PSF_SIZE):
                    img_i = i + ki - pad
                    img_j = j + kj - pad
                    if 0 <= img_i < image_size and 0 <= img_j < image_size:
                        col_idx = img_i * image_size + img_j
                        conv_matrix[row_idx, col_idx] = psf[ki, kj]
    return conv_matrix.tocsr()

# 预计算卷积矩阵
conv_matrix = build_conv_matrix(psf, IMAGE_SIZE)

In [5]:
with pm.Model() as model:
    # 先验分布：每个像素值在 [0, 1] 之间
    x = pm.Uniform("x", 0, 1, shape=(IMAGE_SIZE**2,))
    
    # 模糊过程：矩阵乘法模拟卷积
    blurred = pm.Deterministic("blurred", conv_matrix @ x)
    
    # 似然函数：泊松噪声
    y_obs = pm.Poisson("y_obs", mu=blurred * 10, observed=noisy_image.flatten())

NotImplementedError: Cannot convert <Compressed Sparse Row sparse matrix of dtype 'float64'
	with 71824 stored elements and shape (1024, 1024)> to a tensor variable.

In [ ]:
import pymc.math.